In [50]:
# Declarations
import glob
import json
import os
import numpy as np
import pandas as pd
import re

from os.path import basename

In [51]:
# Functions definitions
def parse(filepath,idval):
    """
    Parse text at given filepath

    Parameters
    ----------
    filepath : str
        Filepath for file to be parsed

    Returns
    -------
    data : pd.DataFrame
        Parsed data

    """

    data = []
    vDSP = 0
    vBRAM = 0
    vFF = 0
    vArea = 0
    # vmS = []
    # vMF = []
    vmS = np.array([])
    vMF = np.array([])
    vCC = 0
    
    with open(filepath, 'r') as file:
        line = file.readline()
        
        while line:
            reg_match = _RegExLib(line)
            
            if reg_match.DSP:
                dsp = reg_match.DSP
                stri, val= line.strip().split(':')
                vDSP = float(val)+vDSP
                
            if reg_match.BRAM:
                bram = reg_match.BRAM
                stri, val= line.strip().split(':')
                vBRAM = float(val)+vBRAM
                
            if reg_match.flip_flop:
                ff = reg_match.flip_flop
                stri, val= line.strip().split(':')
                vFF = float(val)+vFF

            if reg_match.Area:
                area = reg_match.Area
                stri, val= line.strip().split(':')
                vArea = float(val)+vArea
                
            if reg_match.Min_Slack:
                msl = reg_match.Min_Slack
                stri, val= line.strip().split(':')
                vmS = np.append(float(val),vmS)
                
            if reg_match.Max_Freq:
                MFr = reg_match.Max_Freq
                stri, val= line.strip().split(':') 
                vMF = np.append(float(val),vMF)  
            
            if reg_match.Clock_Cycles:
                cc = reg_match.Clock_Cycles
                stri, val= line.strip().split(':')
                valV= val.split()
                vCC = float(valV[0])
            
            line = file.readline()
        
        if vmS.size == 0:
            vmS = np.array([0])

        if vMF.size == 0:
            vMF = np.array([0])
        
        dict_of_data = {
                        'ID_VAL': idval,
                        'DSP': vDSP,
                        'BRAM': vBRAM,
                        'Flip_Flop': vFF,
                        'Area': vArea,
                        'min_Slack': np.min(vmS),
                        'Max_Freq': np.max(vMF),
                        'Clock_Cycles': vCC
                    }
        data.append(dict_of_data)
        
        data = pd.DataFrame(data)
        data.set_index(['ID_VAL','DSP','BRAM','Flip_Flop','Area','min_Slack','Max_Freq','Clock_Cycles'], inplace=True)
        
    return data

In [52]:
# Class definitions
class _RegExLib:
    """Set up regular expressions"""
    _reg_DSP = re.compile('(.*)DSPs: \d(.*)')
    _reg_BRAM = re.compile('(.*)BRAM bitsize: \d(.*)')
    _reg_flip_flop = re.compile('flip-flops in function (.*)')
    _reg_Area = re.compile('(.*)Total estimated area: \d(.*)')
    _reg_Min_Slack = re.compile('(.*)Minimum slack: \d(.*)')
    _reg_Max_Freq = re.compile('(.*)Estimated max frequency (.*)')
    _reg_Clock_Cycles = re.compile('(.*)Total cycles (.*)')

    def __init__(self, line):
        # check whether line has a positive match with all of the regular expressions
        self.DSP = self._reg_DSP.match(line)
        self.BRAM = self._reg_BRAM.match(line)
        self.flip_flop = self._reg_flip_flop.search(line)
        self.Area = self._reg_Area.search(line)
        self.Min_Slack = self._reg_Min_Slack.search(line)
        self.Max_Freq = self._reg_Max_Freq.search(line)
        self.Clock_Cycles = self._reg_Clock_Cycles.search(line)

In [53]:
# DataFrame name json file
nameFile = []
pathFile = []
rootFile = []

tp = 'HW' # HW or SW

path = os.getcwd()
for root, dirs, files in os.walk('../../../SLIDE-x-D001-Artix-7-Recipe/SLIDE-x-RES/RECIPE_DECIMAL'):
    for file in files:
        if file.endswith("cc4csValues.csv"):
            nameFile.append(file)
            pathFile.append(os.path.join(root, file))
            rootFile.append(root)

#indR = nameFile.index("ParMatrixclockCycles.csv")
#nameFile.remove("ParMatrixclockCycles.csv")
#del pathFile[indR]
#del rootFile[indR]

nameFileS = pd.Series(nameFile)
pathFileS = pd.Series(pathFile)
rootFileS = pd.Series(rootFile)

pd.options.display.max_colwidth = 100
print('nameFileS:'+nameFileS)
print('pathFileS:'+pathFileS)
print('rootFileS:'+rootFileS)

dfNF = pd.concat([nameFileS, pathFileS, rootFileS], axis=1)
dfNF.columns = ['FILE_NAME', 'FILE_PATH', 'FILE_ROOT']

#if tp == 'HW':
dfNF = dfNF[dfNF.FILE_ROOT.str.contains('Bambu')]
#if os.name == 'nt':
#    dfNF = dfNF[~dfNF.FILE_ROOT.str.contains('xc7vx690t-3ffg1930-Virtex-7')]
#    dfNF = dfNF[~dfNF.FILE_ROOT.str.contains('xc7z020-1clg484-Zynq-7000')]
##        dfNF = dfNF[~dfNF.FILE_ROOT.str.contains('bellmanford\\Bambu\\results\\long')] # PERCHE' c'è questa cartella long sotto Bambu????
#        dfNF = dfNF[~dfNF.FILE_ROOT.str.contains('bellmanford\\Bambu\\results\\int32_t')] # PERCHE' è vuota di Area file?  
#        dfNF = dfNF[~dfNF.FILE_ROOT.str.contains('bellmanford\\Bambu\\results\\int8_t')] # PERCHE' è vuota di Area file? 
#        dfNF = dfNF[~dfNF.FILE_ROOT.str.contains('bellmanford\\Bambu\\results\\int16_t')] # PERCHE' è vuota di Area file? 
#        dfNF = dfNF[~dfNF.FILE_ROOT.str.contains('selectionsort\\Bambu\\results\\int32_t')] # PERCHE' è vuota di Area file?  
#        dfNF = dfNF[~dfNF.FILE_ROOT.str.contains('selectionsort\\Bambu\\results\\int8_t')] # PERCHE' è vuota di Area file? 
#        dfNF = dfNF[~dfNF.FILE_ROOT.str.contains('selectionsort\\Bambu\\results\\int16_t')] # PERCHE' è vuota di Area file?
#else:
#    dfNF = dfNF[~dfNF.FILE_ROOT.str.contains('xc7vx690t-3ffg1930-Virtex-7')] 
#    dfNF = dfNF[~dfNF.FILE_ROOT.str.contains('xc7z020-1clg484-Zynq-7000')]
##        dfNF = dfNF[~dfNF.FILE_ROOT.str.contains('bellmanford/Bambu/results/long')] # PERCHE' c'è questa cartella long sotto Bambu????
#        dfNF = dfNF[~dfNF.FILE_ROOT.str.contains('bellmanford/Bambu/results/int32_t')] # PERCHE' è vuota di Area file?  
#        dfNF = dfNF[~dfNF.FILE_ROOT.str.contains('bellmanford/Bambu/results/int8_t')] # PERCHE' è vuota di Area file? 
#        dfNF = dfNF[~dfNF.FILE_ROOT.str.contains('bellmanford/Bambu/results/int16_t')] # PERCHE' è vuota di Area file? 
#        dfNF = dfNF[~dfNF.FILE_ROOT.str.contains('selectionsort/Bambu/results/int32_t')] # PERCHE' è vuota di Area file?  
#        dfNF = dfNF[~dfNF.FILE_ROOT.str.contains('selectionsort/Bambu/results/int8_t')] # PERCHE' è vuota di Area file? 
#        dfNF = dfNF[~dfNF.FILE_ROOT.str.contains('selectionsort/Bambu/results/int16_t')] # PERCHE' è vuota di Area file?
#elif tp == 'SW':
#    dfNF = dfNF[~dfNF.FILE_ROOT.str.contains('Bambu')]

0      nameFileS:cc4csValues.csv
1      nameFileS:cc4csValues.csv
2      nameFileS:cc4csValues.csv
3      nameFileS:cc4csValues.csv
4      nameFileS:cc4csValues.csv
                 ...            
125    nameFileS:cc4csValues.csv
126    nameFileS:cc4csValues.csv
127    nameFileS:cc4csValues.csv
128    nameFileS:cc4csValues.csv
129    nameFileS:cc4csValues.csv
Length: 130, dtype: object
0      pathFileS:../../../SLIDE-x-D001-Artix-7-Recipe-Miss/SLIDE-x-RES/RECIPE_DECIMAL/cnt/Bambu/float/o...
1      pathFileS:../../../SLIDE-x-D001-Artix-7-Recipe-Miss/SLIDE-x-RES/RECIPE_DECIMAL/cnt/Bambu/float/o...
2      pathFileS:../../../SLIDE-x-D001-Artix-7-Recipe-Miss/SLIDE-x-RES/RECIPE_DECIMAL/cnt/Bambu/float/o...
3      pathFileS:../../../SLIDE-x-D001-Artix-7-Recipe-Miss/SLIDE-x-RES/RECIPE_DECIMAL/cnt/Bambu/float/o...
4      pathFileS:../../../SLIDE-x-D001-Artix-7-Recipe-Miss/SLIDE-x-RES/RECIPE_DECIMAL/cnt/Bambu/float/o...
                                                      ...                  

In [54]:
pv = dfNF['FILE_PATH'].values
rv = dfNF['FILE_ROOT'].values

for i in range(0,len(dfNF['FILE_PATH'].values)):
    nameFileI = []
    pathFileI = []
    rootFileI = []
    data = pd.DataFrame(columns=['ID_VAL','DSP','BRAM','Flip_Flop','Area','min_Slack','Max_Freq','Clock_Cycles'])
    data.empty
    
    pF = pv[i]
    rF = rv[i]
    dfSI = pd.read_csv(pF, sep=',')
    idval = dfSI['id'].values
    
    if os.name == 'nt':
        spR = pF.split('\\')
        #fileObject = open(rF+"\\"+stri+idval[0]+".txt", "r")
    else:
        spR = pF.split('/')
        #fileObject = open(rF+"/"+stri+idval[0]+".txt", "r")
        
    device = spR[-7]
    board = spR[-6]
    function = spR[-4]
    dataType = spR[-2]
    
    if tp == 'HW':
        stri = "Area_"
    elif tp == 'SW':
        stri = "Size_"
    
    for j in range(0,len(idval)):
        if os.name == 'nt':
            #fileObject = open(rF+"\\"+stri+idval[j]+".txt", "r")
            filepath = rF+"\\"+stri+idval[j]+".txt"
        else:
            #fileObject = open(rF+"/"+stri+idval[j]+".txt", "r")
            filepath = rF+"/"+stri+idval[j]+".txt"
            # print('Filepath:['+str(i)+']['+str(j)+']:'+str(filepath))
            
        dataN = parse(filepath,idval[j])
        if j == 0:
            data = dataN
        else:
            data = data.append(dataN)
    
    if os.name == 'nt':
        data.to_csv(rF+"\\idAreaValuesParam.csv", sep=',')
    else:
        data.to_csv(rF+"/idAreaValuesParam.csv", sep=',')
        
    # print('Filepath:['+str(i)+']:'+str(filepath))
    
    #data = fileObject.read()
    #print(data)   

In [6]:
%debug

> /home/ubuntu/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py(87)_wrapreduction()
     85                 return reduction(axis=axis, out=out, **passkwargs)
     86 
---> 87     return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
     88 
     89 

ipdb> filepath
*** NameError: name 'filepath' is not defined
ipdb> dataN
*** NameError: name 'dataN' is not defined
ipdb> exit


In [8]:
print('OK')

OK
